Computational Assignment 2 Part 1 GWMMs

In [ ]:
import os
print(os.getcwd())  # Printing working directory


c:\Program Files\Microsoft VS Code


Creating a custom simplified GWMM

In [ ]:
# Importing necessary libraries
import cobra
from cobra.io import write_sbml_model, save_json_model

# Creating a new model
model = cobra.Model("Assignment_Model_Complete")

# Defining metabolites (intracellular)
O = cobra.Metabolite('O', compartment='c') # Oxygen
C = cobra.Metabolite('C', compartment='c') # Carbon
N = cobra.Metabolite('N', compartment='c') # Nitrogen
B = cobra.Metabolite('B', compartment='c') # Biomass
P = cobra.Metabolite('P', compartment='c') # Product

O_e = cobra.Metabolite('O_e', compartment='e') # Oxygen external
C_e = cobra.Metabolite('C_e', compartment='e') # Carbon external
N_e = cobra.Metabolite('N_e', compartment='e') # Nitrogen external
B_e = cobra.Metabolite('B_e', compartment='e') # B external
P_e = cobra.Metabolite('P_e', compartment='e') # P external

# Defining reactions with stoichiometry and GPR rules
# Demand reactions (DM) for external metabolites
DM_C = cobra.Reaction('DM_C_e') # Carbon import
DM_C.lower_bound = -1000 # Import to system (negative flux)
DM_C.upper_bound = 0
DM_C.add_metabolites({C_e: -1})

DM_O = cobra.Reaction('DM_O_e')
DM_O.lower_bound = -1000 # Import to system (negative flux)
DM_O.upper_bound = 0
DM_O.add_metabolites({O_e: -1})

DM_N = cobra.Reaction('DM_N_e')
DM_N.lower_bound = -1000 # Import to system (negative flux)
DM_N.upper_bound = 0
DM_N.add_metabolites({N_e: -1})

# Main reactions (v1 to v7)
v1 = cobra.Reaction('v1')
v1.add_metabolites({O_e: -1, O: 1}) # O enters the cell
v1.lower_bound = 0 # Irreversible import only
v1.upper_bound = 1000
v1.gene_reaction_rule = "gene1" # Defining associated gene/genes

v2 = cobra.Reaction('v2')
v2.add_metabolites({C_e: -1, C: 1}) # C enters cell
v2.lower_bound = 0 # Irreversible import only
v2.upper_bound = 1000
v2.gene_reaction_rule = "gene1 or gene2 or gene3" # Defining associated gene/genes

v3 = cobra.Reaction('v3')
v3.add_metabolites({N_e: -1, N: 1}) # N enters cell
v3.lower_bound = 0 # Irreversible import only
v3.upper_bound = 1000
v3.gene_reaction_rule = "gene2 or gene4" # Defining associated gene/genes

v4 = cobra.Reaction('v4') # 2B formation from 0, C and N
v4.add_metabolites({O: -1, C: -1, N: -1, B: 2})
v4.lower_bound = 0 # Irreversible 
v4.upper_bound = 1000
v4.gene_reaction_rule = "(gene5 or gene6) and gene7" # Defining associated gene/genes

v5 = cobra.Reaction('v5')
v5.add_metabolites({C: -1, N: -1, B: 1, P: 1}) # BP from CN
v5.lower_bound = -1000  # Reversible 
v5.upper_bound = 1000
v5.gene_reaction_rule = "gene8 or (gene9 and gene10)" # Defining associated gene/genes

v6 = cobra.Reaction('v6')
v6.add_metabolites({B: -1, B_e: 1}) # B exits from cell
v6.lower_bound = 0 # Irreversible
v6.upper_bound = 1000
v6.gene_reaction_rule = "gene11 and gene12 and gene13" # Defining associated gene/genes
 
v7 = cobra.Reaction('v7')
v7.add_metabolites({P: -1, P_e: 1}) # e export
v7.lower_bound = 0 # Irreversible
v7.upper_bound = 1000
v7.gene_reaction_rule = "gene14" # Defining associated gene/genes

# Sink reactions (SK) for export of metabolites
SK_B = cobra.Reaction('SK_B_e')
SK_B.add_metabolites({B_e: -1}) # Removes B from the environment
SK_B.lower_bound = 0 # Irreversible
SK_B.upper_bound = 1000

SK_P = cobra.Reaction('SK_P_e')
SK_P.add_metabolites({P_e: -1}) # Removes P from the environment
SK_P.lower_bound = 0 # Irreversible
SK_P.upper_bound = 1000

# Adding all reactions to the model
model.add_reactions([DM_C, DM_O, DM_N, v1, v2, v3, v4, v5, v6, v7, SK_B, SK_P])

# Saving the model 
# Specified directory 
save_path = r"\\mfs03\user07\HLSRICE\Documents\Computational\EXAM2\PART_1"

# Save as SBML 
write_sbml_model(model, f"{save_path}\\assignment_model_complete.xml")

# Save as JSON 
save_json_model(model, f"{save_path}\\assignment_model_complete.json")


Restricted license - for non-production use only - expires 2026-11-23


Q2

Restricting the model inputs 

In [ ]:
# Reimporting libraries for reproducibility
import cobra
from cobra.io import write_sbml_model, save_json_model

# Defining the file path for the model
model_path = r"\\mfs03\user07\HLSRICE\Documents\Computational\EXAM2\PART_1\assignment_model_complete.xml"

# Loading the model
model = cobra.io.read_sbml_model(model_path)

# Setting the objective function (optimising v6)
model.objective = "v6" 

# setting the maximum uptake constraints for N, C, and O
# Uptake of N (DM_N_e) with a maximum of 10 mmol/gDW/h
model.reactions.get_by_id("DM_N_e").lower_bound = -10
model.reactions.get_by_id("DM_N_e").upper_bound = 0

# Uptake of C (DM_C_e) with a maximum of 15 mmol/gDW/h
model.reactions.get_by_id("DM_C_e").lower_bound = -15
model.reactions.get_by_id("DM_C_e").upper_bound = 0

# Uptake of O (DM_O_e) with a maximum of 10 mmol/gDW/h
model.reactions.get_by_id("DM_O_e").lower_bound = -10
model.reactions.get_by_id("DM_O_e").upper_bound = 0

# Saving the updated model in both formats (SBML and JSON)
save_path = r"\\mfs03\user07\HLSRICE\Documents\Computational\EXAM2\PART_1"
write_sbml_model(model, f"{save_path}\\assignment_model_updated.xml")  # Save as XML (SBML)
save_json_model(model, f"{save_path}\\assignment_model_updated.json")  # Save as JSON 



No objective coefficients in model. Unclear what should be optimized


Calculating FBA with restricted inputs

In [ ]:
# Reimporting libraries for reproducibility
import cobra
from cobra.io import read_sbml_model

# Loading the updated model using path
model_path = r"\\mfs03\user07\HLSRICE\Documents\Computational\EXAM2\PART_1\assignment_model_updated.xml"
model = read_sbml_model(model_path)

# Running Flux Balance Analysis (FBA)
solution = model.optimize()

# Displaying the flux distribution for all reactions
print("Flux Distribution from FBA:\n")
print(solution.fluxes)

# Saving the flux distribution to CSV
solution.fluxes.to_frame().to_csv(r"\\mfs03\user07\HLSRICE\Documents\Computational\EXAM2\PART_1\FBA_flux_distribution.csv")


Flux Distribution from FBA:

DM_C_e   -10.0
DM_O_e   -10.0
DM_N_e   -10.0
v1        10.0
v2        10.0
v3        10.0
v4        10.0
v5         0.0
v6        20.0
v7         0.0
SK_B_e    20.0
SK_P_e     0.0
Name: fluxes, dtype: float64


Escher maps for visualisation

In [ ]:
# Importing Escher library for escher mapping 
import escher
from escher import Builder

# Specifing path
map_path = r"\\mfs03\user07\HLSRICE\Documents\Computational\EXAM2\PART_1\assignment_map.json"

# Loading the custom map with the optimised model
builder = Builder(
    model=model,              # defining the optimised model
    map_json=map_path,        # defining the custom Escher map that was given from the assignment (template)
    reaction_data=solution.fluxes  # Integrate FBA results into the heatmap to show effects
)

# Display the map
builder



Builder(reaction_data={'DM_C_e': np.float64(-10.0), 'DM_O_e': np.float64(-10.0), 'DM_N_e': np.float64(-10.0), …

Defining colour scale

In [ ]:
# Customise map appearance to maintain consistency over graphs
builder.reaction_scale_preset = 'GaBuRd'  # Blue to red gradient
builder.reaction_scale = [
    {'type': 'min', 'color': '#0000FF', 'size': 10},  # Defining low flux (blue)
    {'type': 'median', 'color': '#FFFF00', 'size': 15},  # defining medium flux (yellow)
    {'type': 'max', 'color': '#FF0000', 'size': 25}   # defining high flux (red)
]

# Hiding certain labels for clarity
builder.hide_secondary_metabolites = True
builder.hide_all_labels = False  # Setting to True hides all labels


Saving

In [ ]:
# Save the Escher map as HTML for visualisation
save_path = r"\\mfs03\user07\HLSRICE\Documents\Computational\EXAM2\PART_1\assignment_custom_escher_map.html"
builder.save_html(save_path)



Q4

Reloading model

In [ ]:
# Define the model path and reloading
model_path = r"\\mfs03\user07\HLSRICE\Documents\Computational\EXAM2\PART_1\assignment_model_updated.xml"

# Function to reload the model to ensure initial conditions for speed and efficiency
def reload_model():
    model = read_sbml_model(model_path)
    return model


GENE1

Reloading model and displaying gene1 deletion

In [ ]:
# Reloading the initial model
model = reload_model()

# Deleting Gene1
model.genes.get_by_id("gene1").knock_out()

# Re-running FBA
solution_gene1 = model.optimize()

# Displaying the FBA results for deletion of Gene1
print("FBA Results after Deleting Gene1:\n")
print(solution_gene1.fluxes)


FBA Results after Deleting Gene1:

DM_C_e   -10.0
DM_O_e     0.0
DM_N_e   -10.0
v1         0.0
v2        10.0
v3        10.0
v4         0.0
v5        10.0
v6        10.0
v7        10.0
SK_B_e    10.0
SK_P_e    10.0
Name: fluxes, dtype: float64


GENE5

Gene5 deletion

In [ ]:
# Reloading the initial model
model = reload_model()

# Deleting Gene5
model.genes.get_by_id("gene5").knock_out()

# Re-running FBA
solution_gene5 = model.optimize()

# Displaying the FBA results for deletion of Gene5
print("FBA Results after Deleting Gene5:\n")
print(solution_gene5.fluxes)



FBA Results after Deleting Gene5:

DM_C_e   -10.0
DM_O_e   -10.0
DM_N_e   -10.0
v1        10.0
v2        10.0
v3        10.0
v4        10.0
v5         0.0
v6        20.0
v7         0.0
SK_B_e    20.0
SK_P_e     0.0
Name: fluxes, dtype: float64


GENE14

Gene14 deletion

In [ ]:
# Reloading the initial model
model = reload_model()

# Deleting Gene14
model.genes.get_by_id("gene14").knock_out()

# Re-running FBA
solution_gene14 = model.optimize()

# Displaying the FBA results for deletion of Gene14
print("FBA Results after Deleting Gene14:\n")
print(solution_gene14.fluxes)


FBA Results after Deleting Gene14:

DM_C_e   -10.0
DM_O_e   -10.0
DM_N_e   -10.0
v1        10.0
v2        10.0
v3        10.0
v4        10.0
v5         0.0
v6        20.0
v7         0.0
SK_B_e    20.0
SK_P_e     0.0
Name: fluxes, dtype: float64


Visualisation

Creating a table

In [ ]:
# TABLE for results
import pandas as pd

# Baseline FBA results before deletion
baseline = {
    "Reaction": ["DM_C_e", "DM_O_e", "DM_N_e", "v1", "v2", "v3", "v4", "v5", "v6", "v7", "SK_B_e", "SK_P_e"],
    "Baseline Flux": [ -10, -10, -10, 10, 10, 10, 10, 0, 20, 0, 20, 0]
}

# Results after gene deletions
results = {
    "Gene1 Deleted": [-10, 0, -10, 0, 10, 10, 0, 10, 10, 10, 10, 10],
    "Gene5 Deleted": [-10, -10, -10, 10, 10, 10, 10, 0, 20, 0, 20, 0],
    "Gene14 Deleted": [-10, -10, -10, 10, 10, 10, 10, 0, 20, 0, 20, 0]
}

# Create a df for table
df = pd.DataFrame(baseline)
df["Gene1 Deleted"] = results["Gene1 Deleted"]
df["Gene5 Deleted"] = results["Gene5 Deleted"]
df["Gene14 Deleted"] = results["Gene14 Deleted"]

# Displaying table
df.style.set_caption("Comparison of FBA Results Before and After Gene Deletions")


,Reaction,Baseline Flux,Gene1 Deleted,Gene5 Deleted,Gene14 Deleted
0,DM_C_e,-10,-10,-10,-10
1,DM_O_e,-10,0,-10,-10
2,DM_N_e,-10,-10,-10,-10
3,v1,10,0,10,10
4,v2,10,10,10,10
5,v3,10,10,10,10
6,v4,10,0,10,10
7,v5,0,10,0,0
8,v6,20,10,20,20
9,v7,0,10,0,0


Escher model

Defining colours

In [ ]:
# Re-defining the colour scale to enure same colours after VSC reload
standard_scale = [
    {'type': 'min', 'color': '#0000FF', 'size': 10},  # low flux (blue)
    {'type': 'median', 'color': '#FFFF00', 'size': 15},  # medium flux (yellow)
    {'type': 'max', 'color': '#FF0000', 'size': 25}   # high flux (red)
]


Building baseline map for comparison

In [ ]:
# Reimporting libraries for reproducibility
import escher
from escher import Builder

# Loading the map for the baseline model
baseline_map = Builder(
    model=model,              # The optimised model
    map_json=map_path,        # The custom Escher map
    reaction_data=solution.fluxes  # The Baseline FBA results
)

# Applying the color scale
baseline_map.reaction_scale = standard_scale
baseline_map.reaction_scale_preset = 'GaBuRd'  # Ensuring blue to red

# Saving Map
baseline_map.save_html(r"\\mfs03\user07\HLSRICE\Documents\Computational\EXAM2\PART_1\baseline_map.html")


GENE1

Gene1 deletion map

In [ ]:
# gene1 map
gene1_map = Builder(
    model=model,              # The model with gene1 deleted
    map_json=map_path,        # custom Escher map
    reaction_data=solution_gene1.fluxes  # Gene1 deleted FBA results
)

# Apply the colour scale
gene1_map.reaction_scale = standard_scale
gene1_map.reaction_scale_preset = 'GaBuRd'

# Save the Gene1 map
gene1_map.save_html(r"\\mfs03\user07\HLSRICE\Documents\Computational\EXAM2\PART_1\gene1_deleted_map.html")


GENE5

Gene5 deletion map

In [ ]:
# gene5 map
gene5_map = Builder(
    model=model,              # The model with gene5 deleted
    map_json=map_path,        # custom Escher map
    reaction_data=solution_gene5.fluxes  # Gene5 deleted FBA results
)

# Apply the colour scale
gene5_map.reaction_scale = standard_scale
gene5_map.reaction_scale_preset = 'GaBuRd'

# Save the Gene5 map
gene5_map.save_html(r"\\mfs03\user07\HLSRICE\Documents\Computational\EXAM2\PART_1\gene5_deleted_map.html")


GENE14

Gene14 deletion map

In [ ]:
# gene14 map
gene14_map = Builder(
    model=model,              # The model with gene14 deleted
    map_json=map_path,        # Your custom Escher map
    reaction_data=solution_gene14.fluxes  # Gene14 deleted FBA results
)

# Apply the colour scale
gene14_map.reaction_scale = standard_scale
gene14_map.reaction_scale_preset = 'GaBuRd'

# Save the Gene14 map
gene14_map.save_html(r"\\mfs03\user07\HLSRICE\Documents\Computational\EXAM2\PART_1\gene14_deleted_map.html")


Q5

In [ ]:
Systematic deletion of each gene to test if any increase v7 flux while keeping v6 flux as optimal

In [ ]:
import cobra
from cobra.io import read_sbml_model

# Loading the model 
model_path = r"\\mfs03\user07\HLSRICE\Documents\Computational\EXAM2\PART_1\assignment_model_updated.xml"

def reload_model():
    model = read_sbml_model(model_path)
    return model

# Function to test gene deletions and track v7 (P_e production)
def test_gene_deletion(gene_id):
    model = reload_model()
    model.genes.get_by_id(gene_id).knock_out()
    solution = model.optimize()
    return solution.fluxes.get("v7"), solution.fluxes.get("v6")

# List of all genes
genes_to_test = ["gene1", "gene2", "gene3", "gene4", "gene5", "gene6", "gene7", "gene8", "gene9", "gene10", "gene11", "gene12", "gene13", "gene14"]

# Dictionary to store the resultss
gene_deletion_results = {}

# Testing each gene deletion
for gene in genes_to_test:
    v7_flux, v6_flux = test_gene_deletion(gene)
    gene_deletion_results[gene] = {"v7_flux": v7_flux, "v6_flux": v6_flux}

# Displaying the results
import pandas as pd
results_df = pd.DataFrame.from_dict(gene_deletion_results, orient="index")
results_df.index.name = "Gene Deleted"
results_df.columns = ["Flux of P_e (v7)", "Flux of B (v6)"]
results_df


,Flux of P_e (v7),Flux of B (v6)
Gene Deleted,,
gene1,10.0,10.0
gene2,0.0,20.0
gene3,0.0,20.0
gene4,0.0,20.0
gene5,0.0,20.0
gene6,0.0,20.0
gene7,10.0,10.0
gene8,0.0,20.0
gene9,0.0,20.0
